In [66]:
import pandas as pd
import ast

In [108]:
df=pd.read_csv(r'C:\Users\Yasaman\Arab_spring_scholarly_attention\Validating\final_annotated.csv')
df2=pd.read_excel(r"C:\Users\Yasaman\Downloads\samples_1127.xlsx")
Map={row['ID']:row['Mentions'] for i,row in df2.iterrows()}
df.loc[(df['SampleGroup'] == 'with_mention_arab') & (df['Mentions'].isna()), 'Mentions'] = df.loc[(df['SampleGroup'] == 'with_mention_arab') & (df['Mentions'].isna()), 'ID'].map(Map)


In [109]:
df['union_annotation'] = df['union_annotation'].apply(ast.literal_eval)
df['intersection_annotation'] = df['intersection_annotation'].apply(ast.literal_eval)
df['Mentions'] = df['Mentions'].apply(ast.literal_eval)


In [110]:
df['annotator_agreement'].value_counts()

annotator_agreement
True     891
False    109
Name: count, dtype: int64

In [111]:
sample_accuracies = df.groupby("SampleGroup").apply(
    lambda x: pd.Series({
        "accuracy_union": (x.apply(lambda row: set(row['Mentions']) == set(row['union_annotation']), axis=1).mean()),
        "accuracy_intersection": (x.apply(lambda row: set(row['Mentions']) == set(row['intersection_annotation']), axis=1).mean())
    })
).reset_index()

sample_accuracies = sample_accuracies.sort_values(by='accuracy_union').reset_index(drop=True)
sample_accuracies

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_21972\3424697931.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_accuracies = df.groupby("SampleGroup").apply(


,SampleGroup,accuracy_union,accuracy_intersection
0,with_mention_arab,0.840000,0.750000
1,with_mention,0.928571,0.775000
2,field_20,0.975000,0.959615


In [105]:
df['Mentions']=df["Mentions"].apply(lambda x: sorted(x))
df['union_annotation']=df["union_annotation"].apply(lambda x: sorted(x))
df['intersection_annotation']=df["intersection_annotation"].apply(lambda x: sorted(x))

In [112]:
sample_accuracies = df.groupby("SampleGroup").apply(
    lambda x: pd.Series({
        "jaccard_union": (
            x.apply(
                lambda row: len(set(row['Mentions']).intersection(set(row['union_annotation']))) /
                            len(set(row['Mentions']).union(set(row['union_annotation'])))
                if len(set(row['Mentions']).union(set(row['union_annotation']))) > 0 else 1,
                axis=1
            ).mean()
        ),
        "jaccard_intersection": (
            x.apply(
                lambda row: len(set(row['Mentions']).intersection(set(row['intersection_annotation']))) /
                            len(set(row['Mentions']).union(set(row['intersection_annotation'])))
                if len(set(row['Mentions']).union(set(row['intersection_annotation']))) > 0 else 1,
                axis=1
            ).mean()
        )
    })
).reset_index()

sample_accuracies = sample_accuracies.sort_values(by='jaccard_union').reset_index(drop=True)
sample_accuracies

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_21972\292121645.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_accuracies = df.groupby("SampleGroup").apply(


,SampleGroup,jaccard_union,jaccard_intersection
0,with_mention_arab,0.905411,0.820729
1,with_mention,0.946131,0.804048
2,field_20,0.976923,0.960577
